In [131]:
# Importing the libraries 
import pandas as pd
import re

In [132]:
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE",
    "PROVIDERID",
    "APPROVINGAGENCYID"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)
print('done')

done


In [140]:
# Set up columns to keep, fields, locations for writing
processedpath = "./D4AD_Standardization/data/processed/"

content_is = "NAME_lookup.csv"

the_df = df # df.sample(n=100, random_state=42)

columns_to_save = ['STANDARDIZEDNAME', 'NAME', 'PROVIDERID',
                    'APPROVINGAGENCYID', 'CIPCODE']
"""
the_df['STANDARDIZED_NAME'] =\
    the_df.NAME.str.split(" - ", n=1)[0]


# So the regex r'\A([^-]+)\s? almost completely works but
# fails in cases of ABC-DEF - NJ and it is more straightforward
# to split on " - " and get take the first result.

# copy over null matches
index = the_df.STANDARDIZED_NAME.isna()
the_df[index].STANDARDIZED_NAME = the_df[index].NAME
"""
"""
the_df.to_csv(rootpath + processedpath + "{}".format(content_is),
              index = False,
              chunksize = 10000,
              columns=columns_to_save)
"""
print('done')

done


In [135]:
# Let's transform this column into a final version...

# 1) we cover the simplest case of hyphenation
the_df['STANDARDIZEDNAME'] = the_df.NAME.str.split(" - ", n=1).str[0]

# 2) then cover cases of 'X-'
regex_pattern = '''
                ^                   # start from beginning
                (.+?                # capture everything non-greedily ...
                    (?:(?!-\s)      # ... except for the '- ', if it's there
                        .)          # and continue to match any character
                *)                  # ... as many times as we can
                '''

the_df.STANDARDIZEDNAME =\
    the_df.STANDARDIZEDNAME.str.extract(regex_pattern, flags=re.VERBOSE)

# 3) Then go after odd static patterns that are commeon
# ... people like to put the color orange, closed in the name of the provider
the_df.STANDARDIZEDNAME =\
    the_df.STANDARDIZEDNAME.str.replace("\(orange\)","", case=False)
the_df.STANDARDIZEDNAME =\
    the_df.STANDARDIZEDNAME.str.replace("closed","", case=False)

# # todo: check the data, maybe 200 cases to get the Jeffrey's interval here
# pd.set_option('display.max_rows', None)
# the_df.sample(n=100)[['STANDARDIZEDNAME', 'NAME']] # see some rows

,STANDARDIZEDNAME,NAME
7272,Raritan Valley Community College,Raritan Valley Community College - Non Credit
18491,Mercer County Community College,Mercer County Community College - Certificate ...
9037,"New Horizons of Central and Northern NJ, Inc.","New Horizons of Central and Northern NJ, Inc.-..."
2708,Mercer County Community College,Mercer County Community College - Certificate ...
4917,C.D.E. Computer Learning Center,C.D.E. Computer Learning Center
17997,Camden County College,Camden County College - Continuing Education
5012,T.H.E. Technologies,T.H.E. Technologies - PCS
5876,American Training School for Medical Professio...,American Training School for Medical Professio...
523,Cortiva Institute,Cortiva Institute - Hoboken - PCS
9519,Southern NJ Chapter of the Appraisal Institute,Southern NJ Chapter of the Appraisal Institute


In [142]:
# ... finally we can write this out as our first complete lookup table
# for the NAME field
the_df.to_csv(rootpath + processedpath + "{}".format(content_is),
              index = False,
              chunksize = 10000,
              columns=columns_to_save)